# Kvalita kódu
Většina poznámek, které nalezneme v tomto repozitáři, se točí kolem otázky jak něco udělat. Text, který právě čteme, má zaměření jiné - věnuje se kvalitě kódu a jeho udržitelnosti. Tj. jde nám o to, jak napsat kód tak, aby se v něm některý náš kolega (anebo třeba i my sami po půl roce) vyznal a aby byl kód co možná nejméně náročný na údržbu. 

# Obsah
- [Virtuální prostředí](#Virtuální-prostředí)  
- [Requirements](#Requirements)  
- [PEP 8](#PEP-8)  
  - [Black](#Black)  
  - [Yapf](#Yapf)  
  - [Pylint](#Pylint)  
  - [Flake8](#Flake8)  
- [Typové anotace a mypy](#Typové-anotace-a-mypy)  
- [Pydantic](#Pydantic)  
- [Docstringy](#Docstringy)  
- [Bandit](#Bandit)
- [Github Actions](#Github-Actions)

## Virtuální prostředí
Člověku nic nebrání nainstalovat si nejprve Python a potom skrze pip i různorodé balíčky. Dřív nebo později ale dojde k situaci, kdy projekt A potřebuje balíček o verzi 0.20 a ne novější, zatímco projekt B stojí a padá s totožným balíčkem, ale ve verzi 1.25 a ne starší. Co s tím potom dělat? V závislosti na úloze, na které programátor zrovna pracuje, balíčky přeinstalovávat? Z toho by byl jen jeden velký zmatek a ještě by se přitom mohly nabořit verze dalších balíčků. Správně řešení spočívá ve využití virtuálních prostředí.  
Virtuální prostředí vytvoříme příkazem (puštěným v příkazovém řádku)
```
python -m venv environment
```
Zde říkáme "hlavní" instalaci pythonu, že chceme pustit balíček [venv](https://docs.python.org/3/library/venv.html) jako skript (proto parametr -m) s parametrem environment. Environment zde nereprezentuje klíčové slovo, ale jméno nového virtuálního prostředí a současně adresáře, který vznikne v pracovním adresáři příkazového řádku. Následně se do nově vytvořeného virtuálního prostředí přepneme příkazem
```
environment\Scripts\activate
```
resp. pokud jsme na Linuxu a nikoli na Windowsech
```
source environment\bin\activate
```
V tento okamžik už se příkazy **python** a **pip** nebudou vázat na python.exe a pip.exe bydlící v instalačním adresáři Pythonu, nýbrž na stejnojmené exe soubory ležící ve virtuálním prostředí. Když budeme nyní instalovat balíčky, uloží se (a budou viditelné v) jen do aktivního virtuálního prostředí a nebudou nijak interferovat s obsahem jiných virtuálních prostředí anebo s balíčky nainstalovanými v rámci základní pythoní instalace. Pokud chceme virtuální prostředí opustit, napíšeme do konzole příkaz
```
deactivate
```
Nikdy do adresáře environment (resp. jeho ekvivalentu, pokud jsme zvolili jiné jméno) explicitně neukládáme vlastní soubory. Onen adresář se totiž neverzuje (je na to moc velký a nic důležitého, co by nebylo jinde, v něm není). I kdybychom chtěli adresář překopírovat či přesunout, raději to nedělejme - lepší je vytvořit nové identické virtuální prostředí (při přesunu by se mohla nabořit vazba s mateřskou instalací).  
Virtuální prostředí mají ze svého principu jednu nevýhodu - pokud nějaký balíček potřebujeme v několika z nich, musíme ho do každého prostředí nainstalovat. To ve výsledku znamená, že se nám po nějaké době nemusí dostávat místo na disku. Proto může být účelné prostředí v nepoužívaných projektech vymazat s tím, že v případě nutnosti ona prostředí znovuvytvoříme s pomocí requirements souborů.

## Requirements
Soubor requirements.txt v minimální variantě obsahuje jména balíčků (co řádek, to jeden balíček), které by měly být pro potřeby určitého projektu nainstalovány. Namísto toho, aby člověk instaloval balíčky ručně jeden po druhém, stačí pustit
```
pip install -r requirements.txt
```
Pozn.: soubor se seznamem balíčků se může jmenovat i jinak, jméno "requirements.txt" je zkrátka jen nejobvyklejší.  
Obvykle chceme, aby virtuální prostředí vytvořené s pomocí requirements.txt bylo totožné jako to, na kterém jsme na našich počítačích původně pracovali.  V takovém případě nám ale nestačí pouze jména balíčků, měli bychom udat i jejich verze. Tj. řádky v requirements.txt by měly vypadat nějak takto:
```
pandas==2.2.2
```
V případě, že by nám z nějakého důvodu stačilo vynucovat minimální verzi balíčku, napíšeme "pandas>=2.0.0", pokud naopak nové verze nechceme, použijeme zápis "pandas<2.0.0".  
Vyvstává tu ale jeden problém. Pakliže bychom do requirements.txt například napsali jen pandy, budou se prerekvizitní balíčky (numpy, six atd.) instalovat podle pandích requirements.txt (či spíše pandího setup.py). Dost možná se tyto balíčky nainstalují v novější verzi, tj. původní a nové virtuální prostředí nebudou stejné a tak hrozí výskyt bugů. Ok, takže si obsah našeho prostředí zafixujeme skrze pip freeze? Tj. do konzole napíšeme
```
pip freeze > requirements.txt
```
Nyní by se ale do našeho nového, dejme tomu produkčního environmentu nainstalovaly i věci typu pytest anebo třeba jupyter, které v produkci potřeba nebudou.  
Na rovinu nyní netuším, zda je lepší možnost 1, možnost 2 anebo zda neexistuje třetí cesta skrze dodatečnou utilitu typu Poetry. Možná je řešením mít krom vývojového prostředí i testovací prostředí, kde budou jen ty opravdu potřebné balíčky pro běh aplikace a do produkčního requirements pak půjde freeze právě tohoto prostředí? 

## PEP 8
Jak velké používat odřádkování (a mají se používat tabulátory či mezerníky)? Jaká by měla být maximální délka řádku? Podle jaké konvence by měly být pojmenovány proměnné? Tyto a mnohé další otázky mohou mít principielně různé odpovědi. V rámci jednoho týmu by jejich řešení mohlo vést k dlouhému dohadování, v globálním pohledu by zase různé předpoklady o tom, jak má co vypadat, mohly vést k ztížené čitelnosti kódu od různých lidí. Aby se zamezilo divergenci v konvencích, vznikl oficiální dokument věnovaný vhodné formální podobě kódu [PEP 8](https://peps.python.org/pep-0008/). Člověk též občas narazí na zmínky o [Google konvencích](https://google.github.io/styleguide/pyguide.html). Ty se možná kdysi od PEP 8 lišily, dnes se ale víceméně jedná o podrobnější rozepsání oblastí v PEP 8 zmíněných.  

No jo, ono je sice užitečné si výše uvedené dokumenty přečíst, ale kdo má při tvorbě a kontrole kódu na vše myslet? Naštěstí existují utility, které nám s takovou prací pomohou. Jedná se o formattery a lintery. Formattery hledají věci, které lze bez znalosti kontextu kódu opravit, a opravu provedou. Mluvíme tu o špatném odřádkování, chybějících či přebývajících mezerách apod. Nejznámějšími formattery jsou **Black** a **Yapf**. Lintery sice chybějící či přebývající mezery taky hledají, nicméně zaměřují se i na nepoužité proměnné či funkce, chybějící importy či nevhodná pojmenování. Změny ale neprovádějí, jen uživateli problémy nahlásí. V oblasti pythoních linterů dominují **Pylint** a **Flake8**.

### Black
[Black](https://black.readthedocs.io/en/stable/) je význačný možnostmi nastavení - téměř žádné nejsou. 
Použití je triviální - v konzoli člověk napíše
```
black muj_skript.py
```
případně
```
black adresar_se_skripty
```
Jednu z mála věcí, které se v Blacku dají změnit, představuje šířka řádku. Pokud nám defaultní velikost 88 nevyhovuje, použijeme parametr -l (resp. --line-length):
```
black adresar_se_skripty -l 100
```

### Yapf
Yapf si můžeme představit jako Black, u kterého se dá změnit úplně všechno.  
Pro základní úpravu jednoho souboru musíme použít parametr -i (resp. --in-place) - bez něj by se výstup Yapfu neuložil do souboru, nýbrž by se pouze vytiskl do konzole.
```
yapf -i muj_skript.py
```
Pokud chceme upravit všechny soubory v adresáři, musíme navíc přidat parametr -r (--recursive)
```
yapf -i -r adresar_se_skripty
```
Defaultně Yapf postupuje podle PEP 8. Chceme-li, aby se pracovalo podle Google stylu, použijeme parametr --style následovaný hodnotou "google":
```
yapf -i --style google muj_skript.py
```
Pokud bychom z nějakého důvodu chtěli defaultní chování specifikovat explicitně, použijeme namísto toho hodnotu "pep8". Můžeme si vytvořit i svůj vlastní styl. Ten popíšeme v konfiguračním souboru (v příkladu pojmenovaném jako .yapfstyle, ale jmenovat se může všelijak). Příklad budiž takovýto:
```
[style]
based_on_style = pep8
column_limit = 40
```
Zde specifikujeme, z jakého stylu vycházíme a následně uvádíme změny - v příkladu maximální šířku řádku rovnou 40 znakům. Jména parametrů jsou k nalezení v [dokumentaci](https://github.com/google/yapf). Při samotném použití napíšeme jméno konfiguráku za parametr --style:
```
yapf -i --style .yapfstyle muj_skript.py
```

### Pylint
Pylintem zkontrolujeme pythoní skript pomocí příkazu
```
pylint muj_skript.py
```
Pylint můžeme vypustit i na celý adresář:
```
pylint adresar_se_skripty
```
Vždy dostaneme report věující se postupně různým okruhům problémů, který je ukončen bodovým hodnocením. Bodové maximum reprezentující z hlediska Pylintu ideální kód se rovná 10, minimum může být v principu v mínus nekonečnu (samozřejmě by se člověk musel hodně snažit, aby skončil pod dejme tomu -10). Též se uživateli ukáže bodové hodnocení předchozího běhu, aby mohl posoudit, jestli se kvalita kódu s provedenými změnami spíše zlepšila či zhoršila.  

Představme si nyní, že bychom Pylint nepoušteli ručně v rámci nějaké ad-hoc kontroly, nýbrž že by byl součástí CICD pipeliny. Jak docílit toho, aby se v případě nízkého skóre nasazovací pipelina zastavila? I když to při ruční práci nevidíme, Pylint krom reportu jako hádám každý proces vrací *exit code*. Nicméně v tomto případě číslo různé od nuly neznamená, že Pylint jako program spadl, nýbrž že ve zkoumaném skriptu nalezl nějaké problémy. A jaké? To je definované kódem určeným s pomocí [této](https://pylint.pycqa.org/en/latest/user_guide/usage/run.html#exit-codes) tabulky. Jestli správně chápu její logiku, tak výskyt aspoň jednoho problému určitého druhu přispívá odpovídajícím číslem z tabulky a tato čísla se ve finále sečtou. Přitom výskyt různých problémů stejného typu už žádný dodatečný příspěvek nepřidává. Tj. například  když bychom v reportu měli tři problémy s konvencí a dva errory, vrátil by se nám *exit code* o velikosti 18. A jak vlastně onen *exit code* zobrazíme? Musíme jako příkaz následující po proběhnutí pylintu ve windosowské konzoli pustit následující:
```
echo %ERRORLEVEL%
```
No jo, jenže občas kód nesplňuje všechny záležitosti na 100%. A asi nedává smysl, aby se nasazení zastavilo kvůli jedné hlášce na konvenci. Proto lze Pylint pustit s parametrem --fail-under=cislo, který zajistí, že se nenulový error code vygeneruje jen když je skóre reportu menší než cislo (defaultně 10). Tj. v praxi to vypadá třeba takto:
```
pylint --fail-under=7 muj_skript.py
```  
Druhou možností, jak se zbavit vlivu nechtěných hlášek, je hlášky určité úrovně (**C**onvention, **W**arning, **E**rror) zkrátka vypnout. To se provede s pomocí parametru "--disable=X", kde X je C, W či E. Pokud chceme vypnout více kategorií, oddělíme jejich písmena čárkou. Tj. provádíme například volání
```python
pylint muj_skript.py --disable=C,W 
```  
Člověk může specifikovat mnohá nastavení v příkazové řádce, to však není ani pohodlné, ani z hlediska produkčního řešení udržitelné (kdo si to má vše pamatovat). Naštěstí lze změny oproti defaultu napsat do konfigurečního souboru. Ten obvykle nese jméno "pylintrc" či ".pylintrc" (existují i jiné možnosti - viz [zde](https://pylint.pycqa.org/en/latest/user_guide/usage/run.html#command-line-options)). Při spuštění pylintu ani není třeba přání použít custom nastavení vyjádřit - pylint hledá konfigurák a pokud ho najde, tak ho použije.  
No jo, ale jak vlastně onen konfigurační soubor vyrobit? Nejsnáze toho dosáhneme exportem defaultního nastavení do souboru a následnými úpravami.
```
pylint --generate-toml-config > .pylintrc
```

### Flake8
Úvodem bych měl podotknout, že narozdíl od doposud diskutovaných nástrojů jsem v praxi zatím Flake8 nikdy nepoužil. Tj. případné nepřesnosti či opomenutí nějaké význačné featury pocházejí z neznalosti, nikoli ze zlého úmyslu.  
Základní použití Flake8 má podobu
```
flake8 muj_skript.py 
```
resp.
```
flake8 muj_adresar
```
Pakliže nebude nalezen žádný problém, v konzoli se žádná hláška neobjeví (pokud pro nás tato skutečnost představuje problém, spustíme Flake8 s parametrem -v). V opačném případě hledíme na výstup ve vstylu
```
muj_skript.py:4:1: F401 'datetime' imported but unused
```
I Flake8 dovoluje uživateli vytvořit si svůj vlastní konfigurační soubor s vlastními pravidly - více viz [zde](https://flake8.pycqa.org/en/latest/user/configuration.html).

## Typové anotace a mypy
Jednou z výhod Pythonu je možnost mít proměnnou v jeden okamžik obsahující datový typ X a za chvíli hostící datový typ Y. Rychle se tak vytvářejí prototypy programů. Jednu z nevýhod Pythonu představuje skutečnost, že je možné mít proměnnou v jeden okamžik obsahující datový typ X a za chvíli hostící datový typ Y. Tím pádem se může do proměnné dostat věc, která tam být nemá. V lepším případě kvůli tomu program spadne, v horším úspěšně běží, ale vrátí nesmysl.  
Řešení tohoto problému spočívá v typových anotacích. Jejich použití v jednoduché funkci si ukážeme na následujícím příkladu:
```python
get_hello_message(user_name:str)->str:
    return f"Hello {user_name}"
```
Ono ":str" říká, že do funkce by měla jít proměnná v podobě textového řetězce. "->str" nám zase naznačuje, že výstupem funkce bude též textový řetězec.  
Musíme zde zdůraznit jednu důležitou věc - Python jako takový typové anotace ignoruje. Ty tak primárně slouží jen jako svého druhu dokumentace. Existují nicméně způsoby, z nichž nejznámější je balíček *mypy*, kterými můžeme typové anotace automaticky kontrolovat.  
Výše jsme si ukázali použití typových anotací pro vstup do a výstup z funkce. Typové anotace lze použít (a opravdu se tak děje, byť fakticky v menší míře) i v "normálním" kódu. Vypadá to nějak takhle:
```python
some_int_variable:int = 4
some_uninitialized_int_variable:int
```
Pro základní datové typy se použijí anotace *int*, *float*, *str* a *bool*. Pokud bychom potřebovali oanotovat bytový řetězec, použijeme anotaci *bytes*.

U kolekcí je situace poněkud složitější a závisí na tom, s jakou verzí Pythonu pracujeme. Pro Python 3.8 a starší se musely typové anotace pro listy, tuply apod. importovat z balíčku *typing*. V příkladu níže si všimněme, že první písmena anotací jsou velká.  
Datové typy elementů v listu či v množině se coby jeden prvek uvednou v hranatých závorkách. Pro slovníky jsou v hranatých závorkách dva elementy - první reprezentuje datový typ klíče, druhý datový typ hodnoty. Nakonec v případě tuplu vložíme do hranatých závorek datový typ každého elementu.
```python
from typing import List, Dict, Tuple

some_list: List[str] = ["Ahoj", "Nazdar"]
some_dict: Dict[int, str] = {1: "Ahoj", 2: "Nazdar"}
some_tuple: Tuple[str,str,int] = ("Ahoj", "Nazdar", 42)
```
V případě, že například v listu může být více datových typů, použijeme *Union*:
```python
from typing import List, Union
some_list: List[Union[str, int]] = ["Ahoj", "Nazdar", 42]
```
Nakonec pokud nějaké proměnná může krom "obvyklé" hodnoty obsahovat i None, aplikujeme *Optional*, do jehož hranatých závorek normální typ vložíme:
```python
some_variable: Optional[str] = "something
```
Poznamenejme, že pokud pracujeme s funkcí, která vrací jen a pouze None, tak se žádným *Optional* neoznačuje - namísto toho prostě napíšeme  *None*:
```python
get_nothing(user_name:str)->None:
    print(f"Hello {user_name})"
```
Pro Python 3.9 a novější se již importování kolekcí dělat nemusí. Níže vidíme, že v takovém případě mají jejich datové typy první písmeno malé.
```python
some_list: list[str] = ["Ahoj", "Nazdar"]
some_dict: dict[int, str] = {1: "Ahoj", 2: "Nazdar"}
some_tuple: tuple[str,str,int] = ("Ahoj", "Nazdar", 42)
```
Od Pythonu 3.10 už není třeba *Union*, jeho roli přebere znak *|*.
```python
some_list: List[str|int] = ["Ahoj", "Nazdar", 42]
```
Nakonec poznamenejme, že "datový typ" pro cokoli (aka defaultní chování Pythonu) nese označení *Any*. Pro použití nějakého objektu (dejme tomu pandího dataframu) můžeme použít jméno onoho objektu včetně zdrojového balíčku, tj. např. "-> pd.DataFrame".  
Existuje i mnoho dalších vzácněji používaných anotací - přehled lze nalézt [zde](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html).

Inu dobrá, svůj kód jsme onanotovali. Jak ale provedeme jeho kontrolu? Jak již bylo řečeno výše, použijeme na to balíček [mypy](https://pypi.org/project/mypy/). Konkrétněji provoláme *mypy* s jménem souboru či adresáře coby parametrem:
```python
mypy muj_skript.py
```
resp.
```python
mypy muj_adresar
```
Občas se pro balíčky třetích stran objeví hláška ve stylu
```
error: Library stubs not installed for "pandas"  [import-untyped]
```
V takovém případě je potřeba doinstalovat balíček se [stub soubory](https://mypy.readthedocs.io/en/stable/stubs.html). Ale popravdě... to jsem nikdy neudělal. Typové anotace mi primárně přijdou užitečné coby pomůcka pro orientaci v kódu a možnost automatických kontrol je pouze třešinka na dortu, kterou v případě větších komplikací mohu oželet.

## Pydantic
Pydantic má podobnou úlohu jako mypy - slouží k validaci typových anotací. Tento balíček zmiňujeme zejména proto, že je součástí známých knihoven jako Langchain, Transformers či FastApi (a docela často ho uvidíme v chybových hláškách při nezdařených instalacích těchto balíčků).  
Pydantic se nezaměřuje na jednoduché proměnné, nýbrž slouží ke kontrole složitějších datových kolekcí. Dalo by se říci, že jeho největší přidaná hodnota tkví ve snadné validace vstupu od uživatele. I když technicky představuje pydantickový datový model pythoní třídu, nejedná se o nahrazení VŠECH tříd v kódu.  
Ukažme si to na příkladu. Napřed si musíme vytvořit model objektu, tj. třídu dědící od *BaseModel*, která bude mít oanotované atributy.

In [1]:
from pydantic import BaseModel

class HamsterModel(BaseModel):
    hamster_id: int
    name: str
    color: str

Následně můžeme vytvořit instanci třídy s tím, že atributy předáme konstruktoru jako keywords arguments.

In [2]:
HamsterModel(hamster_id=3, name="hammy", color="brown")

HamsterModel(hamster_id=3, name='hammy', color='brown')

Pokud bychom totiž předali "normální" poziční argumenty, obdrželi bychom chybu.

In [3]:
HamsterModel(3, "hammy", "brown")

TypeError: BaseModel.__init__() takes 1 positional argument but 4 were given

Chybu obdržíme i v případě, kdy se pokusíme předat argument o špatném datovém typu.

In [4]:
HamsterModel(hamster_id="hodně", name="hammy", color="brown")

ValidationError: 1 validation error for HamsterModel
hamster_id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='hodně', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing

Mohou se vyskytnout případy, kdy by byl kupříkladu string příliš obecný. Pokud může uživatel zadat jen jednu z omezené množiny možností, použijeme *Enum*:

In [5]:
from pydantic import BaseModel
from enum import Enum

class HamsterFood(Enum):
    CARROTS = "carrots"
    PELLETS = "pellets"
    APPLES = "apples"

class HamsterModel(BaseModel):
    hamster_id: int
    name: str
    color: str
    favourite_food: HamsterFood

In [6]:
HamsterModel(hamster_id=3, name="hammy", color="brown", favourite_food="apples")

HamsterModel(hamster_id=3, name='hammy', color='brown', favourite_food=<HamsterFood.APPLES: 'apples'>)

In [7]:
HamsterModel(hamster_id=3, name="hammy", color="brown", favourite_food="blueberries")

ValidationError: 1 validation error for HamsterModel
favourite_food
  Input should be 'carrots', 'pellets' or 'apples' [type=enum, input_value='blueberries', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/enum

Kontrolovat lze i datumy. Ty přitom mohou do validace vstupovat jako stringy o správném formátu (nicméně i podoba datetime.date funguje).

In [8]:
from pydantic import BaseModel
from datetime import date

class HamsterModel(BaseModel):
    hamster_id: int
    name: str
    color: str
    birth_date: date

In [9]:
HamsterModel(hamster_id=3, name="hammy", color="brown", birth_date="2022-10-10")

HamsterModel(hamster_id=3, name='hammy', color='brown', birth_date=datetime.date(2022, 10, 10))

In [10]:
HamsterModel(hamster_id=3, name="hammy", color="brown", birth_date=date(2022, 10, 10))

HamsterModel(hamster_id=3, name='hammy', color='brown', birth_date=datetime.date(2022, 10, 10))

V rámci Pydanticu jsou připraveny i typické složitější datové struktury jako například [mailová adresa](https://docs.pydantic.dev/2.0/usage/types/string_types/#emailstr) (k tomu je pravda potřeba doinstalovat dodatečný balíček [zde](https://pypi.org/project/email-validator/)) či [kladné celé číslo](https://docs.pydantic.dev/2.0/usage/types/number_types/#constrained-integers):

In [11]:
from pydantic import BaseModel, PositiveInt

class HamsterModel(BaseModel):
    hamster_id: int
    name: str
    color: str
    age_in_months: PositiveInt

In [12]:
HamsterModel(hamster_id=3, name="hammy", color="brown", age_in_months=5)

HamsterModel(hamster_id=3, name='hammy', color='brown', age_in_months=5)

In [13]:
HamsterModel(hamster_id=3, name="hammy", color="brown", age_in_months=-5)

ValidationError: 1 validation error for HamsterModel
age_in_months
  Input should be greater than 0 [type=greater_than, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/greater_than

Pokud bychom si s defaultními typy nevystačili, můžeme dodatečné validace specifikovat v rámci [*Field*](https://docs.pydantic.dev/latest/concepts/fields/#field-aliases). Níže uvedené zkratky mají význam "greater than" a "lower than or equal". 

In [14]:
from pydantic import BaseModel, Field

class HamsterModel(BaseModel):
    hamster_id: int
    name: str
    color: str
    age_in_years: int = Field(gt=0, le=3)

In [15]:
HamsterModel(hamster_id=3, name="hammy", color="brown", age_in_years=-1)

ValidationError: 1 validation error for HamsterModel
age_in_years
  Input should be greater than 0 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/greater_than

In [16]:
HamsterModel(hamster_id=3, name="hammy", color="brown", age_in_years=5)

ValidationError: 1 validation error for HamsterModel
age_in_years
  Input should be less than or equal to 3 [type=less_than_equal, input_value=5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/less_than_equal

V rámci textových polí můžeme provádět i kontrolu skrze regulární výrazy:

In [17]:
from pydantic import BaseModel, Field

class HamsterModel(BaseModel):
    hamster_id: int
    name: str
    color: str
    hamster_email: str = Field(pattern=r".+@hamster\.cz$")

In [18]:
HamsterModel(hamster_id=3, name="hammy", color="brown", hamster_email="hammy@hamster.cz")

HamsterModel(hamster_id=3, name='hammy', color='brown', hamster_email='hammy@hamster.cz')

In [19]:
HamsterModel(hamster_id=3, name="hammy", color="brown", hamster_email="hammy@hamster.com")

ValidationError: 1 validation error for HamsterModel
hamster_email
  String should match pattern '.+@hamster\.cz$' [type=string_pattern_mismatch, input_value='hammy@hamster.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/string_pattern_mismatch

## Docstringy
Typové anotace jsou sice fajn věc, nicméně nejlépe funkci funkce (či objektu) vysvětlí normální text. Dávat jednořádkové komentáře do různých sekcí funkce by čtenáři z budoucna komplikovalo pochopení kódu, proto je dobré mít komentářový blok, který se nachází hned pod hlavičkou funkce/metody/třídy. Takovýto text pak nazýváme docstringem.  
Docstringy by měly obsahovat informaci o tom, co (a eventuálně proč a stručně v případě komplikovaných funkcí i jak) funkce dělá. Měly by být popsány vstupy i výstupy a případně i výjimky, které mohou při jejím běhu explicitně vznikat. Z hlediska formátu docstringů se člověk může setkat s prakticky čímkoli, nicméně existují tři dominující konvence - "základní", numpy a google.

### Základní konvence
V rámci této konvence se pod popisem funkce nachází popis vstupů do ní. Přitom pro každý vstup existují dva po sobě jdoucí řádky. V prvním, uvedeném slovy ":param jmeno_parametru:" je tento vstupní parametr popsán. V druhém řádku, který začíná s ":type jmeno_parametru:", je popsán datový typ. Pokud se jedná o parametr nepovinný, objevuje se na tomto řádku i heslo "optional". Podobně to funguje i pro výstup - tehdy řádky začínají s ":return:" a ":rtype:" (bez jména výstupní proměnnné). Více podrobností lze nalézt [zde](https://sphinx-rtd-tutorial.readthedocs.io/en/latest/docstrings.html).

In [3]:
def get_some_nonsense(necessary_number:int, unnecessary_number:int=2)->int:
    """Returns the sum of input numbers

    :param necessary_number: First number.
    :type necessary_number: int
    :param unnecessary_number: Second number.
    :type unnecessary_number: int, optional
    :return: Sum of numbers.
    :rtype: int
    """
    return necessary_number + unnecessary_number

### Numpy konvence
U numpy konvence jsou již jména a datové typy parametrů na stejné řádce.

In [4]:
def get_some_nonsense(necessary_number:int, unnecessary_number:int=2)->int:
    """Returns the sum of input numbers.

    Parameters
    ----------
    necessary_number : int
        First number.
     unnecessary_number : int
         Second number.

    Returns
    -------
    int
        Sum of numbers.
    """
    return necessary_number + unnecessary_number

### Google konvence
Google konvence vypadá jako na klepání do klávesnice úspornější verze numpy konvence. Nutno podotknout, že v nové verzi [Google konvencí](https://google.github.io/styleguide/pyguide.html#383-functions-and-methods) se zde už závorky s datovými typy neobjevují (možná byla tato informace vzhledem k typovým anotacím shledána nadbytečnou?).

In [ ]:
def get_some_nonsense(necessary_number:int, unnecessary_number:int=2)->int:
    """Returns the sum of input numbers.

    Args:
        necessary_number(int): First number.
        unnecessary_number(int): Second number.

    Returns:
        int: Sum of numbers.
    """
    return necessary_number + unnecessary_number

## Bandit
Badit má v principu podobnou funkci jako Pylint či Mypy - slouží ke kontrole kódu. Zaměřuje se přitom výhradně na problémy spojené s jeho bezpečností. Patří sem například hardcodovaná hesla, vyignorované výjimky (velký počet výjimek může indikovat pokus o útok a pokud se takováto věc ani neloguje...) či třeba použití *exec* příkazu. Seznam všech sledovaných věcí nalezneme [zde](https://bandit.readthedocs.io/en/latest/plugins/index.html).  
Samotné použití má následující podobu:
```python
bandit my_script.py
```
přičemž výstup vypadá nějak takto>
```
[main]  INFO    profile include tests: None
[main]  INFO    profile exclude tests: None
[main]  INFO    cli include tests: None
[main]  INFO    cli exclude tests: None
[main]  INFO    running on Python 3.10.11
Run started:2024-07-29 17:33:24.777244

Test results:
>> Issue: [B105:hardcoded_password_string] Possible hardcoded password: '1234hello'
   Severity: Low   Confidence: Medium
   CWE: CWE-259 (https://cwe.mitre.org/data/definitions/259.html)
   More Info: https://bandit.readthedocs.io/en/1.7.9/plugins/b105_hardcoded_password_string.html
   Location: .\my_script.py:16:11
15
16      password = "1234hello"
17      print(password)

--------------------------------------------------

Code scanned:
        Total lines of code: 13
        Total lines skipped (#nosec): 0

Run metrics:
        Total issues (by severity):
                Undefined: 0
                Low: 1
                Medium: 0
                High: 0
        Total issues (by confidence):
                Undefined: 0
                Low: 0
                Medium: 1
                High: 0
Files skipped (0):
```
Pakliže chceme prozkoumat všechny soubory v adresáři, musíme použít parametr -r:
```
bandit -r my_dir
```

## Github Actions
Ono je sice pěkné, že u sebe na lokálu ručně všechny možné i nemožné kontroly spouštíme, ale co když na to jednou zapomeneme a do main větve repozitáře pošleme kód s bugy? Řešení spočívá v automatickém spuštění kontrol při každém pull requestu do mainu či jiné důležité větve. Samo o sobě by se nejednalo o snadnou věc - něco (třeba Jenkins skrz webhooky) by muselo sledovat repozitář, při pull requestu vzít kód, otestovat ho na nějakém pískovišti a na základě výsledku buďto kódu merge dovolit, anebo odmítnout. Naštěstí nic složitého vyrábět  nemusíme - celou tuto funkcionalitu dostáváme (minimálně coby běžní uživatelé) zadarmo v rámci Github Actions.  
Jak na to? V repozitáři si uděláme složku ".github/workflows", do které umístíme soubor "build.yml". Ten bude obsahovat text v následujícím stylu:
```yaml
name: example workflow

on:
  push:
      branches: [ main ]
  pull_request:
      branches: [ main ]


jobs:
  build:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v4
    - name: Create Python environment
      uses: actions/setup-python@v5
      with:
        python-version: 3.10.14
    - name: Install packages
      run: |
        pip install -r requirements.txt
    - name: Do linting checks
      run: |
        pylint --disable=C,W my_script.py
    - name: Run the tests
      run: |
        python -m pytest tests
```
Napřed specifikujeme, pod jakým jménem budou v githubovské záložce "Actions" běhy kontrolního workflow k nalezení. Následně přikazujeme, že se má kontrolní workflow spustit při pushích a pull requestech do větve main. Nakonec zmiňujeme, že testy poběží na nejnovějším Ubuntu s určitou verzí Pythonu, že se mají nainstalovat balíčky z requiremts.txt a že má proběhnout linterování a testy. Pipa za "run:" umožňuje, aby příkazy mohly být na více řádcích.  

V příkladu výše pouštíme celou věc na Linuxu. V případě, že bychom chtěli Windowsy, použijeme v sekci "runs-on" "windows-latest". Nicméně nastává problém - ne pro každou verzi Pythonu je k dispozici windowsovská instalačka. Dostupné kombinace platforem a pythoních verzí lze nalézt [zde](https://raw.githubusercontent.com/actions/python-versions/main/versions-manifest.json).  
V případě, že bychom chtěli naráz testovat více operačních systémů a/nebo verzí Pythonu, musíme yaml soubor upravit následujícím způsobem:
```yaml
name: example workflow

on:
  push:
      branches: [ main ]
  pull_request:
      branches: [ main ]


jobs:
  build:
    runs-on: ${{ matrix.os }}
    strategy:
      matrix:
        os: [windows-latest, ubuntu-latest]
        python-version: [3.12.4, 3.10.14]
    steps:
    - uses: actions/checkout@v4
    - name: Create Python environment
      uses: actions/setup-python@v5
      with:
        python-version: ${{ matrix.python-version }}
    - name: Install packages
      run: |
        pip install -r requirements.txt
    - name: Do linting checks
      run: |
        pylint --disable=C,W hello.py
    - name: Run the tests
      run: |
        python -m pytest -vv --cov=hello test_hello.py
```

Snad ještě jedna věc - v yamlu výše máme uvedené, že se kontrola bude spouštět i po pushi do masteru. To sice ano, ale... Aby Github Actions kontrolu mohlo spustit, tak se napřed musí dostat ke kódu. No a pokud člověk udělá z lokálu push do mainu na Github repozitáři, tak fakticky kontrola proběhne až poté, co už se všechny skripty na mainu zabydlí. Tj. i když kontrola spadne, na mainu špatná verze kódu zůstane.